First we will import the necessary libraries.

In [40]:
import pandas as pd
import os

Next we need to read the data into pandas dataframes. The data was exported from the WCA website in tsv format. The necessary data is contained within 3 seperate tsv files we need to read. "results" contains info on individual completed solves, "persons" contains info on every person who has competed in a WCA competition, and "competitions" contains info on competitions. 

In [60]:
results = pd.read_csv(r'Data/WCA_export_Results.tsv', sep='\t')
print(results.head())

  competitionId eventId roundTypeId  pos  best  average  \
0  LyonOpen2007     333           1   15  1968     2128   
1  LyonOpen2007     333           1   16  1731     2140   
2  LyonOpen2007     333           1   17  2305     2637   
3  LyonOpen2007     333           1   18  2452     2637   
4  LyonOpen2007     333           1   19  2677     2906   

                personName    personId formatId  value1  value2  value3  \
0            Etienne Amany  2007AMAN01        a    1968    2203    2138   
1           Thomas Rouault  2004ROUA01        a    2222    2153    1731   
2  Antoine Simon-Chautemps  2005SIMO01        a    3430    2581    2540   
3           Irène Mallordy  2007MALL01        a    2715    2452    2868   
4       Marlène Desmaisons  2007DESM01        a    2921    3184    2891   

   value4  value5 regionalSingleRecord regionalAverageRecord personCountryId  
0    2139    2108                  AfR                   AfR   Cote d_Ivoire  
1    2334    2046                  N

In [43]:
persons = pd.read_csv("WCA_export_Persons.tsv", sep="\t")
print(persons.head())

   subid              name       countryId gender          id
0      1     Jozsef Borsos          Serbia      m  1982BORS01
1      1  Roland Brinkmann         Germany      m  1982BRIN01
2      1   Julian Chilvers  United Kingdom      m  1982CHIL01
3      1  Jessica Fridrich             USA      f  1982FRID01
4      2  Jessica Fridrich  Czech Republic      f  1982FRID01


In [44]:
competitions = pd.read_csv("WCA_export_Competitions.tsv", sep="\t")
print(competitions.head())

                             id                               name  \
0                 100Merito2018                   100º Mérito 2018   
1    100YearsRepublicAnkara2023     100 Years Republic Ankara 2023   
2  100YearsRepublicIstanbul2023   100 Years Republic İstanbul 2023   
3      100YilMBACubeWeekend2023     100. Yıl MBA Cube Weekend 2023   
4    10AniversarioGuatemala2023  Décimo Aniversario Guatemala 2023   

         cityName  countryId  \
0  Santarém, Pará     Brazil   
1          Ankara     Turkey   
2        İstanbul     Turkey   
3        İstanbul     Turkey   
4  Guatemala City  Guatemala   

                                         information  \
0  Qualquer pessoa que resolva o Cubo Mágico em m...   
1  [TR] Kayıt sırasında adınızı ve soyadınızı lüt...   
2  [TR] Kayıt sırasında adınızı ve soyadınızı lüt...   
3  [TR] Kayıt sırasında adınızı ve soyadınızı lüt...   
4  Ésta es una competencia conmemorativa en honor...   

                                               ve